In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import sys
import os

In [3]:
process_dirs = [""]

for root, dirs, files in os.walk("./data"):
    for dir in dirs:
        if dir in process_dirs:
            print("Processing", dir)
            for root2, dirs2, files2 in os.walk(os.path.join(root, dir)):
                images = []
                image_filenames = []
                for file in files2:
                    img = cv.imread(os.path.join(root2, file))
                    # Find the smallest rect bounds for the image
                    # its a black shape inside
                    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                    
                    # Gaussian blur
                    gray = cv.GaussianBlur(gray, (3, 3), 0)
                    
                    _, thresh = cv.threshold(
                        gray, 100, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
                    )
                    
                    # Manual filter threshold using numpy
                    thresh = np.where(gray < 150, 0, 255).astype(np.uint8)             
                    
                    # Inverse image
                    thresh = cv.bitwise_not(thresh)
                     
                    # Detect countours and find an outer bound
                    contours, hierarchy = cv.findContours(
                        thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE
                    )
                    
                    # Find the biggest contour
                    max_area = 0
                    max_contours = []
                    for contour in contours:
                        area = cv.contourArea(contour)
                        if area > max_area:
                            max_area = area
                            max_contours.append(contour)
                        
                        if max_area == 0:
                            continue
                        
                        area_variance = abs(1 - (area / max_area))
                        print(f"Area variance: {area_variance}, Area: {area}, Max Area: {max_area}")
                        if area_variance < 0.8 and area_variance > 0:
                            max_contours.append(contour)
                            
                    def combine_bounding_boxes(rects):
                        if not rects:
                            return None

                        min_x = float('inf')
                        min_y = float('inf')
                        max_x = float('-inf')
                        max_y = float('-inf')

                        for (x, y, w, h) in rects:
                            min_x = min(min_x, x)
                            min_y = min(min_y, y)
                            max_x = max(max_x, x + w)
                            max_y = max(max_y, y + h)

                        combined_x = min_x
                        combined_y = min_y
                        combined_w = max_x - min_x
                        combined_h = max_y - min_y

                        return (combined_x, combined_y, combined_w, combined_h)
                    
                    bounds = [cv.boundingRect(contour) for contour in max_contours]
                    x, y, w, h = combine_bounding_boxes(bounds)
                    
                    thresh = cv.bitwise_not(thresh)
                    
                    # debug = cv.cvtColor(thresh, cv.COLOR_GRAY2BGR)
                    
                    # # Draw the bounding box
                    # for x_, y_, w_, h_ in bounds:
                    #     cv.rectangle(debug, (x_, y_), (x_ + w_, y_ + h_), (0, 255, 0), 2)

                    # cv.rectangle(debug, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    # plt.imshow(debug)
                    # plt.show()
                    
                    # Crop
                    thresh = thresh[y:y+h, x:x+w]    
                    
                    images.append(thresh)
                    image_filenames.append(file)
                    
                    # plt.imshow(thresh, cmap="gray")
                    # plt.show()
                    
                # create sub dir 
                os.makedirs(os.path.join(root2, "cropped"), exist_ok=True)

                for i, image in enumerate(images):
                    cv.imwrite(os.path.join(root2, "cropped", image_filenames[i]), image)
                

Processing 7
Area variance: 0.0, Area: 552.0, Max Area: 552.0
Area variance: 0.0, Area: 314.0, Max Area: 314.0
Area variance: 0.9888535031847133, Area: 3.5, Max Area: 314.0
Area variance: 0.0, Area: 551.5, Max Area: 551.5
Area variance: 0.0, Area: 389.0, Max Area: 389.0
Area variance: 0.9215938303341902, Area: 30.5, Max Area: 389.0
Area variance: 0.0, Area: 426.0, Max Area: 426.0
Area variance: 0.0, Area: 415.0, Max Area: 415.0
Area variance: 0.9337349397590361, Area: 27.5, Max Area: 415.0
Area variance: 0.0, Area: 1.5, Max Area: 1.5
Area variance: 0.0, Area: 468.5, Max Area: 468.5
Area variance: 0.0, Area: 451.0, Max Area: 451.0
Area variance: 0.8292682926829268, Area: 77.0, Max Area: 451.0
Area variance: 0.0, Area: 269.0, Max Area: 269.0
Area variance: 0.0, Area: 378.0, Max Area: 378.0
Area variance: 0.9907407407407407, Area: 3.5, Max Area: 378.0
Area variance: 0.0, Area: 431.0, Max Area: 431.0
Area variance: 0.0, Area: 437.5, Max Area: 437.5
Area variance: 0.984, Area: 7.0, Max Area